In [ ]:
# Safira Nur Fauziah
# Universitas Dian Nuswantoro

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
# Untuk Balance Data
from imblearn.under_sampling import RandomUnderSampler

# Decision tree, XGBoost dan SVM
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

# Metrics 

build a classifier model to help Bank Talenta Rakyat to gauge their customers efficiently and effectively also optimize the process of screening customer profiles that apply for credit that indicate the chances of borrowers defaulting (cannot repay their loan in more than 59 days)

# Data Cleaning / Data Preparation

## Perempuan

In [ ]:
female = pd.read_csv("/content/drive/MyDrive/Celerates/Project Akhir/application_history_f.csv", sep=';')

female.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5008808,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
1,5008809,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
2,5008810,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
3,5008811,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
4,5008812,Perempuan,Tidak,Ya,0,283500,Pensioner,PG,D,MH,1,0,0,0,NaN,1,62,1001


In [ ]:
print(female.shape)

(294360, 18)


## Laki-laki

In [ ]:
male = pd.read_csv("/content/drive/MyDrive/Celerates/Project Akhir/application_history_m.csv", sep=';')

male.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
2,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3
3,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2
4,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2


In [ ]:
print(male.shape)

(144096, 18)


## Credit History Data

In [ ]:
credit = pd.read_csv("/content/drive/MyDrive/Celerates/Project Akhir/credit_history.csv")

credit.head()

,Id_customer,Overdue
0,5001711,Tidak memiliki pinjaman
1,5001711,0
2,5001711,0
3,5001711,0
4,5001712,0


In [ ]:
credit['Overdue'].value_counts()

0                          821230
Tidak memiliki pinjaman    208755
1                           10936
5                            1677
2                             849
3                             317
4                             222
Name: Overdue, dtype: int64

In [ ]:
# credit historynya diambil max aja baru digabung

In [ ]:
# Ganti value tidak memiliki pinjaman jadi 0
credit['Overdue'] = credit['Overdue'].replace(['Tidak memiliki pinjaman'], '0')

# Ambil maksimum overdue dari setiap Borrowers
g = credit.groupby(['Id_customer'])['Overdue'].transform('max')
credit = credit[(credit['Overdue'] == g)]

credit.head()

,Id_customer,Overdue
0,5001711,0
1,5001711,0
2,5001711,0
3,5001711,0
4,5001712,0


In [ ]:
# Masukin pas sebelum di handling

In [ ]:
# Ubah value overdue
credit['Overdue'] = np.where(credit['Overdue']=='0', 0, 1)

In [ ]:
credit['Overdue'].value_counts()

0    902030
1     11504
Name: Overdue, dtype: int64

## Combine 3 file

1. Borrowers dikatakan memiliki badscore jika tidak bisa membayar pinjaman diatas 30 hari.
2. Untuk Credit history data, ambil maksimum overdue dari setiap Borrowers.
3. Nasabah yang digunakan untuk membuat model hanya yang memiliki history pinjaman(overdue) ---tidak memiliki pinjaman- diklasifikasikan jadi 0 1--

In [ ]:
combine = pd.concat([male, female])

combine.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
2,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3
3,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2
4,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2


In [ ]:
combine['JK'].value_counts()

Perempuan    294360
Laki-laki    144096
Name: JK, dtype: int64

In [ ]:
combine.shape

(438456, 18)

In [ ]:
# Yg bener how nya pakenya inner atau left join harusnya onnya pake customer id

df = pd.merge(combine, credit, on=['Id_customer'], how='inner')

In [ ]:
df.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
2,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
3,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3,0
4,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3,0


In [ ]:
df.shape

(676364, 19)

In [ ]:
df.isna().sum()

Id_customer                 0
JK                          0
KepemilikanMobil            0
KepemilikanProperti         0
JmlAnak                     0
Pendapatan                  0
TipePendapatan              0
TingkatPendidikan           0
StatusKeluarga              0
TipeRumah                   0
FlagMobile                  0
FlagWorkPhone               0
FlagPhone                   0
Email                       0
Pekerjaan              212213
JmlAnggotaKeluarga          0
Age                         0
Experience                  0
Overdue                     0
dtype: int64

In [ ]:
df.isnull().mean() * 100

Id_customer             0.000000
JK                      0.000000
KepemilikanMobil        0.000000
KepemilikanProperti     0.000000
JmlAnak                 0.000000
Pendapatan              0.000000
TipePendapatan          0.000000
TingkatPendidikan       0.000000
StatusKeluarga          0.000000
TipeRumah               0.000000
FlagMobile              0.000000
FlagWorkPhone           0.000000
FlagPhone               0.000000
Email                   0.000000
Pekerjaan              31.375561
JmlAnggotaKeluarga      0.000000
Age                     0.000000
Experience              0.000000
Overdue                 0.000000
dtype: float64

In [ ]:
df[df.isna().any(axis=1)]

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
2,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
491,5008951,Laki-laki,Tidak,Ya,1,225000,Bekerja,G,M,MH,1,0,0,0,NaN,3,28,8,0
492,5008951,Laki-laki,Tidak,Ya,1,225000,Bekerja,G,M,MH,1,0,0,0,NaN,3,28,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676341,5148679,Perempuan,Tidak,Tidak,0,121500,Asosiasi komersial,G,M,MH,1,0,0,0,NaN,2,52,17,1
676345,5149055,Perempuan,Tidak,Ya,0,112500,Asosiasi komersial,G,M,MH,1,1,1,0,NaN,2,43,7,1
676346,5149055,Perempuan,Tidak,Ya,0,112500,Asosiasi komersial,G,M,MH,1,1,1,0,NaN,2,43,7,1
676347,5149056,Perempuan,Tidak,Ya,0,112500,Asosiasi komersial,G,M,MH,1,1,1,0,NaN,2,43,7,1


In [ ]:
df.shape

(676364, 19)

In [ ]:
# Drop NaN

df = df.dropna(subset=['Pekerjaan'], inplace=False)

In [ ]:
df.duplicated().sum()

439158

In [ ]:
# Drop data duplikat

data = df.drop_duplicates(keep='last')

In [ ]:
data.shape

(24993, 19)

In [ ]:
# Simpen dataset buat diolah di Machine Learning
data.to_csv('df_ml.csv', header=True, index=False)

## Change Value Dataset


In [ ]:
# Ganti nama

# Tingkat Pendidikan
data['TingkatPendidikan'] = data['TingkatPendidikan'].replace(['PG'], 'Post Graduate')
data['TingkatPendidikan'] = data['TingkatPendidikan'].replace(['G'], 'Graduate')
data['TingkatPendidikan'] = data['TingkatPendidikan'].replace(['UG'], 'Undergraduate')

<ipython-input-29-c1a3656d9e47>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TingkatPendidikan'] = data['TingkatPendidikan'].replace(['PG'], 'Post Graduate')
<ipython-input-29-c1a3656d9e47>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TingkatPendidikan'] = data['TingkatPendidikan'].replace(['G'], 'Graduate')
<ipython-input-29-c1a3656d9e47>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
data.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
32,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,Graduate,M,MH,1,0,0,0,Security staff,2,59,3,0
38,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,M,MH,1,1,1,1,Accountants,2,46,2,0
81,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,M,MH,1,1,1,1,Accountants,2,46,2,0
89,5008819,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,M,MH,1,0,0,0,Laborers,2,49,3,0
98,5008820,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,M,MH,1,0,0,0,Laborers,2,49,3,0


In [ ]:
# Status Keluarga
data['StatusKeluarga'] = data['StatusKeluarga'].replace(['M'], 'Menikah')
data['StatusKeluarga'] = data['StatusKeluarga'].replace(['NM'], 'Belum Menikah')
data['StatusKeluarga'] = data['StatusKeluarga'].replace(['D'], 'Cerai')

<ipython-input-31-ab03cd439a36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['StatusKeluarga'] = data['StatusKeluarga'].replace(['M'], 'Menikah')
<ipython-input-31-ab03cd439a36>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['StatusKeluarga'] = data['StatusKeluarga'].replace(['NM'], 'Belum Menikah')
<ipython-input-31-ab03cd439a36>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [ ]:
data.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
32,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,Graduate,Menikah,MH,1,0,0,0,Security staff,2,59,3,0
38,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,MH,1,1,1,1,Accountants,2,46,2,0
81,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,MH,1,1,1,1,Accountants,2,46,2,0
89,5008819,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,MH,1,0,0,0,Laborers,2,49,3,0
98,5008820,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,MH,1,0,0,0,Laborers,2,49,3,0


In [ ]:
# Tipe Rumah
data['TipeRumah'] = data['TipeRumah'].replace(['RA'], 'Sewa Apartemen')
data['TipeRumah'] = data['TipeRumah'].replace(['MH'], 'Rumah Pribadi')
data['TipeRumah'] = data['TipeRumah'].replace(['MA'], 'Apartemen Pribadi')
data['TipeRumah'] = data['TipeRumah'].replace(['PH'], 'Rumah Orang Tua')
data['TipeRumah'] = data['TipeRumah'].replace(['OA'], 'Apartemen Kantor')

<ipython-input-33-a0448deb055e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TipeRumah'] = data['TipeRumah'].replace(['RA'], 'Sewa Apartemen')
<ipython-input-33-a0448deb055e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TipeRumah'] = data['TipeRumah'].replace(['MH'], 'Rumah Pribadi')
<ipython-input-33-a0448deb055e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [ ]:
data.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
32,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,Graduate,Menikah,Rumah Pribadi,1,0,0,0,Security staff,2,59,3,0
38,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,Rumah Pribadi,1,1,1,1,Accountants,2,46,2,0
81,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,Rumah Pribadi,1,1,1,1,Accountants,2,46,2,0
89,5008819,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,Rumah Pribadi,1,0,0,0,Laborers,2,49,3,0
98,5008820,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,Rumah Pribadi,1,0,0,0,Laborers,2,49,3,0


In [ ]:
# Overdue
data['Overdue'] = data['Overdue'].replace([0], 'Tepat Waktu')
data['Overdue'] = data['Overdue'].replace([1], 'Tidak Tepat Waktu')

<ipython-input-35-0454f6b5043d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Overdue'] = data['Overdue'].replace([0], 'Tepat Waktu')
<ipython-input-35-0454f6b5043d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Overdue'] = data['Overdue'].replace([1], 'Tidak Tepat Waktu')


In [ ]:
data.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
32,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,Graduate,Menikah,Rumah Pribadi,1,0,0,0,Security staff,2,59,3,Tepat Waktu
38,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,Rumah Pribadi,1,1,1,1,Accountants,2,46,2,Tepat Waktu
81,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,Rumah Pribadi,1,1,1,1,Accountants,2,46,2,Tepat Waktu
89,5008819,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,Rumah Pribadi,1,0,0,0,Laborers,2,49,3,Tepat Waktu
98,5008820,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,Rumah Pribadi,1,0,0,0,Laborers,2,49,3,Tepat Waktu


In [ ]:
data['Overdue'].value_counts()

Tepat Waktu          21958
Tidak Tepat Waktu     3035
Name: Overdue, dtype: int64

In [ ]:
# Data Boolean
data['FlagMobile'] = data['FlagMobile'].replace([1], 'Ya')
data['FlagMobile'] = data['FlagMobile'].replace([0], 'Tidak')
data['FlagWorkPhone'] = data['FlagWorkPhone'].replace([1], 'Ya')
data['FlagWorkPhone'] = data['FlagWorkPhone'].replace([0], 'Tidak')
data['FlagPhone'] = data['FlagPhone'].replace([1], 'Ya')
data['FlagPhone'] = data['FlagPhone'].replace([0], 'Tidak')
data['Email'] = data['Email'].replace([1], 'Ya')
data['Email'] = data['Email'].replace([0], 'Tidak')

<ipython-input-38-a81eb8e4dc69>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FlagMobile'] = data['FlagMobile'].replace([1], 'Ya')
<ipython-input-38-a81eb8e4dc69>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FlagMobile'] = data['FlagMobile'].replace([0], 'Tidak')
<ipython-input-38-a81eb8e4dc69>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [ ]:
data.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
32,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,Graduate,Menikah,Rumah Pribadi,Ya,Tidak,Tidak,Tidak,Security staff,2,59,3,Tepat Waktu
38,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,Rumah Pribadi,Ya,Ya,Ya,Ya,Accountants,2,46,2,Tepat Waktu
81,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,Post Graduate,Menikah,Rumah Pribadi,Ya,Ya,Ya,Ya,Accountants,2,46,2,Tepat Waktu
89,5008819,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,Rumah Pribadi,Ya,Tidak,Tidak,Tidak,Laborers,2,49,3,Tepat Waktu
98,5008820,Laki-laki,Ya,Ya,0,135000,Asosiasi komersial,Graduate,Menikah,Rumah Pribadi,Ya,Tidak,Tidak,Tidak,Laborers,2,49,3,Tepat Waktu


In [ ]:
# Simpen dataset buat diolah di Tableau
data.to_csv('df_tableau.csv', header=True, index=False)